# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [67]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [68]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [69]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)



235.0

In [70]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [71]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

## 1. What test is appropriate for this problem? Does CLT apply?

A two-sample test would be appropriate since we are comparing two sets of data.
The central limit theorem (CLT) would apply because the sample distribution can be approximated as normal, random, and independent. The distribution is approximated as normal since the sample size n > 30. The data can be considered independent since the sample n <10% of the population size. 

## 2. What are the null and alternate hypotheses?

To test whether race has a significant impact on the rate of callbacks for resumes, we will assume the following null and alternate hypotheses.

$H_{o}$ : The proportion of black and white labeled names that receive callbacks are equal. ($p_b = p_w$)

$H_{a}$ : The proportion of black and white labeled names that receive callbacks are not equal. ($p_b \neq p_w$)

## 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [72]:
w = data[data.race=='w']
b = data[data.race=='b']

### bootstrap method

In [73]:
#construct arrays of the two datasets
w_callbacks =  np.array(w['call'])
b_callbacks =  np.array(b['call'])

#Calculate the portions of callbacks for each dataset
p_w_callbacks = np.sum(w_callbacks)/ len(w_callbacks)
p_b_callbacks = np.sum(b_callbacks)/ len(b_callbacks)

#calculate empirical difference
empirical_diff = p_w_callbacks - p_b_callbacks

#Concat the two datasets
bw_callbacks = np.concatenate((w_callbacks,b_callbacks))

#generate shifted arrays
w_callback_shifted = w_callbacks + (np.mean(bw_callbacks)-np.mean(w_callbacks))
b_callback_shifted = b_callbacks + (np.mean(bw_callbacks)-np.mean(b_callbacks))

In [74]:
# Define a function that resamples the data and computes the summary statistic 
#computed from each sample

def bootstrap_replicate_1d(data, func):
    a = np.random.choice(data, size=len(data))
    return func(a)

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.zeros(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)   

    return bs_replicates

In [75]:
#resample the data 10000 times and make a sampling distribution of the sample mean
#for black and white callbacks
w_rep = draw_bs_reps(w_callback_shifted, np.mean, size=10000)
b_rep =draw_bs_reps(b_callback_shifted, np.mean, size=10000)

# Get replicates of difference of means
replicate_difference = w_rep - b_rep

#calculate the margin of error: critical value *std
margin_error = 1.96 * np.std(replicate_difference)
print ('margin of error = ',margin_error)

#Find a confidence interval of 95% : window between [2.5, 97.5] percentile
conf_interval = np.percentile(replicate_difference, [2.5,97.5])
print('confidence interval = ',conf_interval)

# Compute and print p-value: p
p = np.sum(replicate_difference >= empirical_diff) / len(replicate_difference)
print('p-value =', p)


margin of error =  0.015260371205274934
confidence interval =  [-0.01560577  0.01478437]
p-value = 0.0


### Frequentist statistical approach

In [76]:
#calculate porportion average between b and w callbacks
p_average = (p_b_callbacks + p_w_callbacks) / 2

#calculate difference in stds
std_difference = np.sqrt( (( 2 * p_average) * (1 - p_average))/ (len(w_callbacks)))

#calculate the difference in porportions and then calculate z_score
p_difference = p_w_callbacks - p_b_callbacks
z_score = (p_difference - 0 ) / std_difference

#calculate the margin of error for frequential test: critical value * std
margin_error_ft = 1.96 * std_difference
print ('margin of error = ',margin_error_ft)

#Find a confidence interval of 95% for frequential test: mean +/- margin or error
conf_interval_ft = np.array([-1, +1]) * margin_error_ft
print('confidence interval = ',conf_interval_ft)

#calculate p-value from z-score and print
p_value = stats.norm.sf(z_score) * 2
print('p-value =', round(p_value,3))

#print proportion difference
print ('p_w - p_b = ', p_difference)

margin of error =  0.015281912310894095
confidence interval =  [-0.01528191  0.01528191]
p-value = 0.0
p_w - p_b =  0.032032854209445585


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

## 4. Write a story describing the statistical significance in the context of the original problem.

I performed a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes. The statistical analysis was performed using the bootstrap and frequential statistical method. To perform these methods, the null hypothesis was that race had no impact on callback rate. Therefore population proportions of black and white sounding names should have a similar rate ($p_w = p_b$ or $p_w - p_b = 0$). The alternative hypothesis was that the race of the name affected the callback rate so the population proportions would not be equal ($p_w \neq p_b$ or $p_w - p_b \neq 0$). 

From the frequential method, the observed difference in proportions show that $p_w - p_b \approx 0.032$ which shows that $p_w \neq p_b$. From the p-values calculated from both test methods, we can confidently reject the null hypothesis since the p-value $\approx 0$ which is less that the significance level of 0.05. Thus concluding, that race does play a role in callback rates.



## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis done was to prove that race played a role in rate of callbacks and to reject the assumption that it did not. But although we proved that it was a factor, there may be other factors that were not taken into consideration. So we cannot confidently say that it is in fact the most important factor in callback success, unless we have tested the other factors as well in a similar manner. After testing all other possible factors using statistical analysis, we can compare the results and confidently say whether race is the most important factor.